In [1]:
from pipeline import Pipeline, Process, Postprocess, Preprocess
import numpy as np
import pandas as pd

In [2]:
class MyPreProcess(Preprocess):
    process_desc = "Exemple de pre-process -> ne fait rien"
    # prends en param la liste d'images
    def run(self, images)
        pass


class MyProcess(Process):
    process_desc = "Exemple de process -> ne fait rien"
    def run(self, images):
        pass
    
    
class MyPostProcess(Postprocess):
    process_desc = "Exemple de post-process -> ne fait rien"
    def run(self, images):
        pass

In [3]:
pipeline = Pipeline()
process_array = np.array([MyPreProcess()
                          ,MyProcess()
                          ,MyPostProcess()
                          ,MyPreProcess(verbose=0)])
pipeline.add_processes(process_array)

Preprocess :  Exemple de pre-process -> ne fait rien
Process :  Exemple de process -> ne fait rien
Postprocess :  Exemple de post-process -> ne fait rien
Exemple de pre-process -> ne fait rien a été ajouté.
Exemple de process -> ne fait rien a été ajouté.
Exemple de post-process -> ne fait rien a été ajouté.
Exemple de pre-process -> ne fait rien a été ajouté.


In [4]:
pipeline.print_process()

Exemple de pre-process -> ne fait rien
Exemple de pre-process -> ne fait rien
Exemple de process -> ne fait rien
Exemple de post-process -> ne fait rien


# LAYOUT SEATGURU

In [269]:
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np

data_path = "/home/sid2019-1/Documents/M2_DATA_INTERPROMO/"
layout = "LAYOUT SEATGURU/"

images = [data_path + layout + img for img in os.listdir(data_path + layout)]
IMAGE_NAMES = [img for img in os.listdir(data_path + layout)]

def hasNumbers(inputString):
    return any(char.isdigit() for char in inputString)

def longestValue(inputList):
    max_len = 0
    for i, el in enumerate(inputList):
        if len(el) > max_len:
            max_len = i
    return inputList[max_len]

def get_relevant_aircraft_builders(proportion_min=0.02, proportion_max=0.75):
    # retrieve aircraft types
    aircraft_builders = np.unique([a_type.split(' ')[0].lower() for a_type in pd.read_csv(data_path + 'SEATGURU_INFO_AIRCRAFT.csv', sep=';')['Aircraft_Type'].tolist() if len(a_type.split(' ')) > 1])
    
    # remove aircraft_builder that have Numerics in it
    aircraft_builders = [a_builder for a_builder in  aircraft_builders if not hasNumbers(a_builder) and len(a_builder) > 2]
    
    # remove aircraft with too low occurences in the "IMAGES_NAMES" list
    relevant_aircraft_builders = {}
    for a_builder in aircraft_builders:
        proportion = sum([1for img_n in image_names if a_builder in img_n.lower()]) / len(IMAGE_NAMES)
        if proportion > proportion_min and proportion < proportion_max:
            relevant_aircraft_builders[a_builder] = proportion

    # remove non aircradft builder that remain
    del relevant_aircraft_builders['airlines']
    del relevant_aircraft_builders['irbus']
    del relevant_aircraft_builders['airways']
    del relevant_aircraft_builders['ays']

    
    # sort by proportion
    relevant_aircraft_builders = {k: v for k, v in sorted(relevant_aircraft_builders.items(), key=lambda item: item[1])}
    return relevant_aircraft_builders

def get_airline_aircraft_builder_pos(image_names, aircraft_builders, airlines, aircraft_ref):
    # Initialisation of dict
    dictio_airlines_aircraft_builders = []
        
    for image_name in image_names:
        dictio_airlines_aircraft_builders.append({
            'image_name': image_name,
            'aircraft_builder': '', 
            'position_aircraft_builder': -1, 
            'airline': '',
            'aircraft_ref': []
        })
        last_el = len(dictio_airlines_aircraft_builders) - 1
        # initiliase position
        position = -1
        # a little pre-process to clean-up image name
        img_inf = image_name.lower().split('.')[0].split('_')
        for i, item in enumerate(img_inf):
            for a_builder in aircraft_builders:
                # check if the image contains the aircraft builder
                if a_builder == item:
                    # add the aircraft builder in the image name
                    position = i
                    dictio_airlines_aircraft_builders[last_el]['aircraft_builder'] = a_builder
                    
        if position == -1: dictio_airlines_aircraft_builders[last_el]['aircraft_builder'] = 'not_relevant_aircraft_builders'
        dictio_airlines_aircraft_builders[last_el]['position_aircraft_builder'] = position
        
        # add airlines deduce by the image name
        for airline in airlines:
            if airline + '_' in image_name.lower():
                dictio_airlines_aircraft_builders[last_el]['airline'] = airline
                
        # add aircraft_ref
        for a_ref in aircraft_ref:
            if a_ref in  image_name.lower():
                 dictio_airlines_aircraft_builders[last_el]['aircraft_ref'].append(a_ref)
        # take the longest str element in the list of ref
        dictio_airlines_aircraft_builders[last_el]['aircraft_ref'] = longestValue(dictio_airlines_aircraft_builders[last_el]['aircraft_ref'])
    
    return dictio_airlines_aircraft_builders

def main_layout_seatguru():
    relevant_aircraft_builders = get_relevant_aircraft_builders()
    airlines = pd.read_csv(data_path + 'SEATGURU_INFO_AIRCRAFT.csv', sep=';')['Airline_name'].unique()
    airlines = [airline.replace('-','_') for airline in airlines] 

    aircraft_ref = np.unique([a_type.lower() for a_type in pd.read_csv(data_path + 'SEATGURU_INFO_AIRCRAFT.csv', sep=';')['Aircraft_Type'].tolist()])
    aircraft_ref = np.unique([t for text in aircraft_ref  for t in text.split(' ') if hasNumbers(t)])
    
    dictio_airlines_aircraft_builders = get_airline_aircraft_builder_pos(IMAGE_NAMES, relevant_aircraft_builders, airlines, aircraft_ref)
    return pd.DataFrame(dictio_airlines_aircraft_builders)

df_layout_seatguru = main_layout_seatguru()
df_layout_seatguru.head()

,aircraft_builder,aircraft_ref,airline,image_name,position_aircraft_builder
0,boeing,777-300er,qatar_airways,Qatar_Airways_Boeing_777-300ER_plane737.svg.png,2
1,airbus,a319-100,aegean_airlines,Aegean_Airlines_Airbus_A319-100_plane1.svg.png,2
2,boeing,737-800,korean_air,Korean_Air_Boeing_737-800_A_plane601.jpg,2
3,not_relevant_aircraft_builders,q400,united,United_Airlines_Q400_A_plane1005.svg.png,-1
4,airbus,a319,qatar_airways,Qatar_Airways_Airbus_A319_V2_plane742.svg.png,2


In [ ]:
# LAYOUT SEATMAESTRO

In [275]:
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np

data_path = "/home/sid2019-1/Documents/M2_DATA_INTERPROMO/"
layout = "LAYOUT SEATMAESTRO/"

images = [data_path + layout + img for img in os.listdir(data_path + layout)]
IMAGE_NAMES = [img for img in os.listdir(data_path + layout)]

def hasNumbers(inputString):
    return any(char.isdigit() for char in inputString)

def longestValue(inputList):
    max_len = 0
    for i, el in enumerate(inputList):
        if len(el) > max_len:
            max_len = i
    return inputList[max_len]

def get_relevant_aircraft_builders(proportion_min=0.02, proportion_max=0.75):
    # retrieve aircraft types
    aircraft_builders = np.unique([a_type.split(' ')[0].lower() for a_type in pd.read_csv(data_path + 'SEATGURU_INFO_AIRCRAFT.csv', sep=';')['Aircraft_Type'].tolist() if len(a_type.split(' ')) > 1])
    
    # remove aircraft_builder that have Numerics in it
    aircraft_builders = [a_builder for a_builder in  aircraft_builders if not hasNumbers(a_builder) and len(a_builder) > 2]
    
    # remove aircraft with too low occurences in the "IMAGES_NAMES" list
    relevant_aircraft_builders = {}
    for a_builder in aircraft_builders:
        proportion = sum([1 for img_n in image_names if a_builder in img_n.lower()]) / len(IMAGE_NAMES)
        if proportion > proportion_min and proportion < proportion_max:
            relevant_aircraft_builders[a_builder] = proportion

    # remove non aircradft builder that remain
    del relevant_aircraft_builders['airlines']
    del relevant_aircraft_builders['irbus']
    del relevant_aircraft_builders['airways']
    del relevant_aircraft_builders['ays']

    
    # sort by proportion
    relevant_aircraft_builders = {k: v for k, v in sorted(relevant_aircraft_builders.items(), key=lambda item: item[1])}
    return relevant_aircraft_builders

def get_airline_aircraft_builder_pos(image_names, aircraft_builders, airlines, aircraft_ref):
    # Initialisation of dict
    dictio_airlines_aircraft_builders = []
        
    for image_name in image_names:
        dictio_airlines_aircraft_builders.append({
            'image_name': image_name,
            'aircraft_builder': '', 
            'position_aircraft_builder': -1, 
            'airline': '',
            'aircraft_ref': []
        })
        last_el = len(dictio_airlines_aircraft_builders) - 1
        # initiliase position
        position = -1
        # a little pre-process to clean-up image name
        img_inf = image_name.lower().split('.')[0].split('_')
        for i, item in enumerate(img_inf):
            for a_builder in aircraft_builders:
                # check if the image contains the aircraft builder
                if a_builder == item:
                    # add the aircraft builder in the image name
                    position = i
                    dictio_airlines_aircraft_builders[last_el]['aircraft_builder'] = a_builder
                    
        if position == -1: dictio_airlines_aircraft_builders[last_el]['aircraft_builder'] = 'not_relevant_aircraft_builders'
        dictio_airlines_aircraft_builders[last_el]['position_aircraft_builder'] = position
        
        # add airlines deduce by the image name
        for airline in airlines:
            if airline + '_' in image_name.lower():
                dictio_airlines_aircraft_builders[last_el]['airline'] = airline
                
        # add aircraft_ref
        for a_ref in aircraft_ref:
            if a_ref in  image_name.lower():
                 dictio_airlines_aircraft_builders[last_el]['aircraft_ref'].append(a_ref)
        # take the longest str element in the list of ref
#         dictio_airlines_aircraft_builders[last_el]['aircraft_ref'] = longestValue(dictio_airlines_aircraft_builders[last_el]['aircraft_ref'])
    
    return dictio_airlines_aircraft_builders

def main_layout_seatguru():
    relevant_aircraft_builders = get_relevant_aircraft_builders()
    airlines = pd.read_csv(data_path + 'SEATGURU_INFO_AIRCRAFT.csv', sep=';')['Airline_name'].unique()
#     airlines = [airline.replace('-','_') for airline in airlines] 

    aircraft_ref = np.unique([a_type.lower() for a_type in pd.read_csv(data_path + 'SEATGURU_INFO_AIRCRAFT.csv', sep=';')['Aircraft_Type'].tolist()])
    aircraft_ref = np.unique([t for text in aircraft_ref  for t in text.split(' ') if hasNumbers(t)])
    
    dictio_airlines_aircraft_builders = get_airline_aircraft_builder_pos(IMAGE_NAMES, relevant_aircraft_builders, airlines, aircraft_ref)
    return pd.DataFrame(dictio_airlines_aircraft_builders)

df_layout_seatguru = main_layout_seatguru()
df_layout_seatguru.head()

,aircraft_builder,aircraft_ref,airline,image_name,position_aircraft_builder
0,not_relevant_aircraft_builders,"[1, 2, 200, 3, 320, 320-200, a320, a320-200]",,qatar-airways-airbus-a320-200-132pax.png,-1
1,not_relevant_aircraft_builders,"[2, 200, 3, 330, 330-200, a330, a330-200]",,etihad-airways-airbus-a330-200.png,-1
2,not_relevant_aircraft_builders,"[-800, 2, 3, 7-800, 737, 737-800, 8, 800, v3]",,japan-airlines-jal-boeing-b737-800-v30v32.png,-1
3,not_relevant_aircraft_builders,"[2, 3, 340, 4, 6, a340, a340-600, v2]",,iberia-airbus-a340-600-v2.png,-1
4,not_relevant_aircraft_builders,"[787, 787-8, 8]",,air-europa-boeing-b787-8.png,-1
